In [ ]:
import numpy as np

np.set_printoptions(suppress=True)

# Define 7 predefined 4x4 matrices
matrix_1 = np.array([[1, 0, 0, 0],
                     [0, -1, 0, 0],
                     [0, 0, -1, 156.4],
                     [0, 0, 0, 1]])

matrix_2 = np.array([[1, 0, 0, 0],
                     [0, 0, -1, 5.4],
                     [0, 1, 0, -128.4],
                     [0, 0, 0, 1]])

matrix_3 = np.array([[1, 0, 0, 0],
                     [0, -1, 0, -410],
                     [0, 0, -1, 0],
                     [0, 0, 0, 1]])

matrix_4 = np.array([[1, 0, 0, 0],
                     [0, 0, -1, 208.4],
                     [0, 1, 0, -6.4],
                     [0, 0, 0, 1]])

matrix_5 = np.array([[1, 0, 0, 0],
                     [0, 0, 1, 0],
                     [0, -1, 0, -105.9],
                     [0, 0, 0, 1]])

matrix_6 = np.array([[1, 0, 0, 0],
                     [0, 0, -1, 105.9],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1]])

matrix_7 = np.array([[-1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, -1, -61.5],
                     [0, 0, 0, 1]])

# Manual multiplication of the matrices in one line

result = np.dot(np.dot(np.dot(np.dot(np.dot(np.dot(np.dot(np.eye(4), matrix_1), matrix_2), matrix_3), matrix_4), matrix_5), matrix_6), matrix_7)
result


array([[  -1. ,    0. ,    0. ,    0. ],
       [   0. ,   -1. ,    0. ,    1. ],
       [   0. ,    0. ,    1. , 1176.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [ ]:
import numpy as np

np.set_printoptions(suppress=True)

# Definitions of the functions poe and fk_poe

def poe(q, a, rot, joint_type):
    T = np.eye(4)
    for i in range(len(q)):
        j = len(q) - i - 1
        if joint_type[j] == "R":
            rot_i = np.array([[         0, -rot[j][2],  rot[j][1]],
                              [ rot[j][2],          0, -rot[j][0]],
                              [-rot[j][1],  rot[j][0],          0]])
            rot_mat_i = np.eye(3) + np.sin(q[j]) * rot_i + (1 - np.cos(q[j])) * (rot_i @ rot_i)
        elif joint_type[j] == "P":
            rot_i = np.zeros(3)
            rot_mat_i = np.eye(3)

        if joint_type[j] == "R" and j > 0:
            Sv = -np.cross(rot[j], a[j])
        elif joint_type[j] == "R" and j == 0:
            Sv = np.array([0, 0, 0])
        elif joint_type[j] == "P":
            Sv = a[j]

        lin_vec_i = ((np.eye(3) * q[j]) + ((1 - np.cos(q[j])) * rot_i) + ((q[j] - np.sin(q[j])) * (rot_i @ rot_i))) @ Sv
        trans_mat_i = np.column_stack([rot_mat_i, lin_vec_i.T])
        trans_mat_i = np.vstack((trans_mat_i, np.array([0, 0, 0, 1])))

        T = trans_mat_i @ T

    return T

def fk_poe(q, a, rot, joint_type, M):
    T = poe(q, a, rot, joint_type)
    trans_mat = T @ M
    R = trans_mat[:3,:3]
    p = trans_mat[:3, 3]
    return trans_mat, R, p

if __name__ == "__main__":
    # Joint angles/positions
    t1 = 0
    t2 = np.pi/2
    t3 = np.pi/2
    t4 = np.pi/2
    t5 = 0
    t6 = 0
    t7 = 0

    q = [t1, t2, t3, t4, t5, t6, t7]

    # Parameters for the robot
    L1 = 156.4
    L2 = 128.4
    L3 = 410
    H1 = -5.4
    L4 = 208.4
    H2 = 6.4
    L5 = 105.9
    L6 = 105.9
    L7 = 61.5

    # Updated a, rot, and jt lists
    a = np.array([[    0, 0,     0],
                  [    0, 0,   L1],
                  [    0, H1, L1+L2],
                  [0, H1, L1+L2+L3],
                  [0, H1+H2, L1+L2+L3+L4],
                  [0, H1+H2, L1+L2+L3+L4+L5],
                  [0, H1+H2, L1+L2+L3+L4+L5+L6]])

    rot = np.array([[0, 0, 0],
                    [0, 0, 1],
                    [0, 1, 0],
                    [0, -1, 0],
                    [0, 0, -1],
                    [0, -1, 0],
                    [0, 0, -1]])

    jt = ["R", "R", "R", "R", "R", "R", "R"]  # All revolute joints

    # Updated M matrix to consider the additional links
    M = np.array([[-1, 0, 0, 0],
                  [0, -1, 0, H1+H2],
                  [0, 0, 1, L1+L2+L3+L4+L5+L6+L7],
                  [0, 0, 0, 1]])

    ans = fk_poe(q, a, rot, jt, M)

    print(ans[0])



[[ -0.    1.    0.   -1. ]
 [ -1.   -0.    0.  410. ]
 [  0.    0.    1.  766.5]
 [  0.    0.    0.    1. ]]
